![workshops](images/codete_workshops.jpg)

# Slack bot - setup RTM environment

In this section we will create Anna, our HR assistant bot in Slack. This section covers the following topics:
- setting up Slack for bots,
- setting up Trello for Anna,
- explanation between RTM and Events API,
- use Anna to handle the messages

<p align="center">
  <img src="images/hr_bot_logo_S.png" />
</p>

## Setup Slack account for RTM

It's good to create a new slack workspace, but you can use also a slack account where you have the admin priviliges. 
Follow the instructions to setup your Slack account for RTM API:
1. Go to www.slack.com and **register**.
2. Create a new **workspace**.
3. Click **Browser** in the top.
4. Browser for **Bots** and choose it.
5. **Add configuration**
6. Fill out the fields: 
    - Customize Name
    - Full Name
7. Copy the **API Token** and into a variable SLACK_TOKEN.
8. **Save Integration**.

In [ ]:
SLACK_TOKEN = ""

Let's try use Anna. We use the SlackClient package. 

In [ ]:
from slackclient import SlackClient

sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="test", as_user=False) 
else:
    print("Connection failed")

In [ ]:
sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="Hello. I'm Anna, your HR assistant. How can I help?", as_user=True)
else:
    print("Connection failed")

### Exercise 1: Identify with who the bot is getting the message from and get channel id by name. Get also the channel lists by name

Implement a util function ```get_user_map()```. As parameter we get an instance of ```SlackClient```. Use it to get the user lists by ```id``` as a map ```{'id':'name'}```. Find the proper method in the Slack API documentation: [Slack API docs](https://api.slack.com/methods). Use it to display the channel and user names.

In [ ]:
def get_user_map(sc):
    users = {}
    # put your code here
    return users

In [ ]:
def get_channel_id_by_name(sc, channel_name):
    channel_id = ""
    # put your code here
    return channel_id

In [ ]:
import time

sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
      channel_id = get_channel_id_by_name(sc, "general")
      last_message_id = 0
      while True:
           response = sc.api_call("channels.history", channel="C76LMNLFP", count=1,latest="")
           if response['ok']:
                msg = response['messages'][0]
                if msg['ts'] != last_message_id:
                     if 'user' in msg:
                          #put your code here
                          print(msg["user"]+": "+ msg["text"])
                          last_message_id = msg['ts']
           time.sleep(5)       
else:
    print("Connection failed") 

## Trello setup

We need to create a Trello board and get all the keys needed. To do so, we should do the following:
1. Register at [Trello](trello.com).
2. Go to [Trello API keys](https://trello.com/app-key).
3. Copy the API key (top) and secret key (bottom) into below variables.
4. Click to generate a token (a link in the top) and copy it to the variables below.

In [ ]:
API_KEY = ""
API_SECRET_KEY = ""
TOKEN = ""
TOKEN_SECRET = ""

Now, we can test our Trello connection and check what kind of boards do we have.

In [ ]:
from trello import TrelloClient

client = TrelloClient(
    api_key=API_KEY,
    api_secret=API_SECRET_KEY,
    token=TOKEN,
    token_secret=TOKEN_SECRET
)

boards = client.list_boards()
print(boards)

You can setup the boards manually or use the code below to setup all HR pipeline boards.

In [ ]:
def setup_trello_board():
    recruitment_board = None
    for board in boards:
        if board.name == "Recruitment":
            recruitment_board= board
    if recruitment_board == None:
        recruitment_board = client.add_board("Recruitment")
    lists_to_create = ['Prescreening','Before interview','After interview','Finished']
    
    #trello_lists = recruitment_board.all_list()
    
    finished_list     = recruitment_board.add_list(lists_to_create[3])
    after_list        = recruitment_board.add_list(lists_to_create[2])
    before_list       = recruitment_board.add_list(lists_to_create[1])
    prescreening_list = recruitment_board.add_list(lists_to_create[0])        
    
    return recruitment_board, prescreening_list, before_list, after_list, finished_list

# run once
board, prescreening_list, before_list, after_list, finished_list = setup_trello_board()

Let's test it by adding a new card to a board.

In [ ]:
prescreening_list.add_card("John Kowalski - Data Scientist","Some description goes here")

### Exercise 2: Combine RTM API with Trello. If the message does have only the list word in message, we should list the cards in prescreening list on Slack.

Hint: you can use some methods available in the [doc](https://github.com/sarumont/py-trello).

In [ ]:
sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
    #channel_id = get_channel_id_by_name(sc, "general")
    last_message_id = 0
    while True:
        response = sc.api_call("channels.history", channel="C76LMNLFP", count=1,latest="")
        if response['ok']:
            msg = response['messages'][0]
            if msg['ts'] != last_message_id:
                if 'user' in msg:
                    # use prescreening_list variable here
                    pass
        time.sleep(5)       
else:
    print("Connection failed") 

## Setup Slack for Events API

Follow the instructions to setup your Slack account for RTM API:
1. Go to your workspace and click **Build** and next **Start Building**.
2. Fill out the **App Name** and choose the Workspace. Next, click on **Create app**.
3. Go to **Event Subscriptions** and turn it on. Save changes.
4. Go to **Bot Users** and fill out the fields. You can turn the online status. Save changes.
5. Go to **OAuth and Permissions** and copy the **Bot User OAuth Access Token** value.
6. Go back to **Event Subscriptions** and subscribe to Workspace Events for **reaction_added**.
7. In Request URL fill as your ngrok URL and add at the end /slack/events. Use the code below to validate the URL.
8. Go back to **Basic Information**. Fill out the App name and give an app icon (optional). Copy the **verification token**.
9. Go to the top and make sure you have changes on **Event Subscriptions**, **Bots** and **Permissions**.
10. Install your app.

In [ ]:
VERIFICATION_TOKEN = ""

In [ ]:
from flask import Flask, request
from slackeventsapi import SlackEventAdapter

app = Flask(__name__)
@app.route("/slack/events",methods=['GET','POST'])
def authorize():
  output = request.get_json()
  return output["challenge"]

if __name__ == "__main__":
  app.run(port=3000)

Now we can start a new instance of our server to handle the actions.

In [ ]:
app = Flask(__name__)

slack_events_adapter = SlackEventAdapter(VERIFICATION_TOKEN, "/slack/events", app)

@slack_events_adapter.on("reaction_added")
def reaction_added(event):
 emoji = event.get("event").get("reaction")
 print(emoji)

if __name__ == "__main__":
  app.run(port=3000)

### Exercise 3: Use the Events API and RTM API and extend the  previous exercise with to handle the messages with Events API and respond with RTM API. 

Hints: 
1. Change the event subscriptions and reinstall the app.
2. Use **Bot User OAuth Access Token** as the SLACK_TOKEN for SlackClient.

In [ ]:
### put your code here